<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2021notebooks/2021_1112Olivetti_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
date: 2021_1112
source: https://qiita.com/takubb/items/7d45ae701390912c7629

---



In [10]:
import platform
isColab = True if platform.system() == 'Linux' else False
if isColab:
    !pip install japanize_matplotlib > /dev/null 2>&1
    !pip install torchsummary > /dev/null 2>&1
    !pip install torchviz > /dev/null 2>&1

import matplotlib.pyplot as plt
import japanize_matplotlib
%config InlineBackend.figure_format = 'retina'
%matplotlib inline    

<center>
<img src="https://komazawa-deep-learning.github.io/assets/Neocognitron.svg" width="49%">
<img src="https://komazawa-deep-learning.github.io/assets/Fukushima.jpeg" width="19%"><br/>
<font size="+2" color="blue">ネオコグニトロン (Fukushima, 1979)</font><br/>
</center>    

<center>
<img src="https://komazawa-deep-learning.github.io/assets/1998LeCun_Fig2_CNN.svg" width="66%"><br/>
<font size="+2" color="blue">LeNet5(LeCun, 1998)</font><br/>
</center>    

<center>
<img src="https://komazawa-deep-learning.github.io/assets/2012AlexNet_2.svg" width="66%"><br/>
<!-- <img src="https://miro.medium.com/max/2812/1*bD_DMBtKwveuzIkQTwjKQQ.png" width="66%"><br/> -->
<font size="+2" color="blue">アレックスネット (Krizensky, et al., 2012)</font><br/>
</center>

<center>
<img src="https://komazawa-deep-learning.github.io/assets/imagenet_result2017.png" width="49%"><br/>
<font size="+2" color="blue">年毎のイメージネットコンテストの結果，縦軸は優勝チームのエラー率</font><br/>
</center>


<!-- https://komazawa-deep-learning.github.io/assets/2019si_conv-demo.mp4 -->
<!-- 
<video width="49%" markdown="0" controls>
<source src="https://komazawa-deep-learning.github.io/assets/2019si_conv-demo.mp4" type="video/mp4" markdown="0" >
</video>
-->

<p>
<center>
<video controls loop>
<source src="https://komazawa-deep-learning.github.io/assets/2019si_conv-demo.mp4" type="video/mp4" style="width:84%">
</video>
</center>
</p>

## 1. CNNの構成

CNN は以下のいずれかの層から構成される：

* 畳込み層
* プーリング層
* 完全結合層（通常のニューラルネットワークと正確に同じもの，CNN では最終 1 層または最終 1,2 層に用いる）

入力信号はパラメータの値が異なる活性化関数によって非線形変換される。 畳込み層とプーリング層と複数積み重ねることで多層化を実現し，深層ニューラルネットワークとなる。

## 2. 畳み込み層

`onv2d` のパラメータ: 
    * in_channels
    * out_channels
    * kernel_size
    * stride
    * padding
    * dilation


* 畳込み層のパラメータは学習可能なフィルタの組
* 全フィルタは空間的に（幅と高さに沿って）小さくなる
* フィルタは入力信号の深さと同一
* 第1層のフィルタサイズは例えば 5×5×3（5 画素分の幅，高さ，と深さ 3（３原色の色チャンネル）
* **ストライド**: 各層の順方向の計算は入力信号の幅と高さに沿って各フィルタを水平または垂直方向へスライド
フィルタの各値と入力信号の特定の位置の信号との内積（ドット積）。
入力信号に沿って水平，垂直方向にフィルタをスライド
各空間位置でフィルタの応答を定める 2 次元の活性化地図が生成される
学習の結果獲得されるフィルタの形状には、方位検出器，色ブロッブ，生理学的には視覚野のニューロンの応答特性に類似
* 上位層のフィルタには複雑な視覚パタンに対応する表象が獲得される
* 各畳込み層全体では学習すべき入力信号をすべて網羅するフィルタの集合が形成される
* 各フィルタは相異なる 2 次元の活性化地図を形成
* 各フィルタの応答特性とみなすことが可能な活性化地図
* フィルタの奥行き次元に沿って荷重総和を計算し、出力信号を生成


1. 深さ数(フィルタ数)
2. ストライド幅
3. ゼロパディング

#### 1. 出力層ニューロン数のことを出力層の 深さ 数と呼ぶハイパーパラメータである。深さ数とはフィルタ数（カーネル数）とも呼ばれる。第 1 畳込み層が生画像であれば，奥行き次元を構成する各ニューロンによって種々の方位を持つ線分(エッジ検出細胞)や色ブロッブのような特徴表現を獲得可能となる。入力の同じ領域を 深さ列 とするニューロン集団を ファイバ ともいう。
#### 2. フィルタを上下左右にずらす幅を ストライド幅 と呼ぶ。ストライド幅が 1 ならフィルタを 1 画素ずつ移動することを意味する。ストライドが 2ならフィルタは一度に 2 画素ずつジャンプさせる。ストライド幅が大きければ入力信号のサンプリング間隔が大きく広がることを意味する。ストライド幅が大きくなれば上位層のニューロン数は減少する。
#### 3. 入力の境界上の値をゼロで埋め込むことがある。これを ゼロパディング という。ゼロパディングの量はハイパーパラメータである。ゼロパディングにより出力層ニューロンの数を制御できる。下位層の空間情報を正確に保存するには入力と出力の幅，高さは同じである必要がある。


### プーリング

CNN では，連続する畳込み層間にプーリング層を挿入するのが一般的。プーリング層の役割は，空間次元の大きさに減少させることである。パラメータ，すなわち計算量を減らし，過学習を制御できる。プーリング層は入力の各深さ毎に独立して動作する。最大値のみをとり他の値を捨てることを マックスプーリング と呼ぶ。サイズが 2x2 のフィルタによるプーリング層では，入力の深さごとに つのダウンサンプルを適用し、幅と高さに沿って2ずつ増やして75％の情報を破棄する。この場合 4 つの数値のうち最大値を採用することになる。

<center>
<img src="https://komazawa-deep-learning.github.io/assets/cnn/maxpool.jpeg" width="49%"><br/>
</center>

一般的なダウンサンプリング演算は マックスプーリング である。図では ストライド 2 すなわち 4 つの数値の中の最大値
平均プーリング. マックスプーリングではなく L2正則化プーリング を行う場合もある。平均プーリングは歴史的な意味あいがあるがマックスプーリングの方が性能が良いとの報告がある。ある画像位置には物理的に一つの値だけが存在するという視覚情報処理が仮定すべき外界の物理的制約を反映していると文学的に解釈することも可能である。

<center>
<img src="https://komazawa-deep-learning.github.io/assets/cnn/pool.jpeg" width="49%"><br/>
</center>

In [ ]:
import torch
import torchvision

alexnet = torchvision.models.alexnet(pretrained=True)

import torchsummary
#torchsummary.summary(alexnet, input_size=(3,224,224))

import torchviz
x = torch.rand([1,3,255,255])
y = alexnet.forward(x)
torchviz.make_dot(y.mean(), params=dict(alexnet.named_parameters()))

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import random
import numpy as np
import matplotlib.pyplot as plt

# from tqdm import tqdm  #コマンドラインで実行するとき
from tqdm.notebook import tqdm  # jupyter で実行するとき

# リソースの選択（CPU/GPU）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 乱数シード固定（再現性の担保）
def fix_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed = 42
fix_seed(seed)

# データローダーのサブプロセスの乱数の seed を固定
def worker_init_fn(worker_id):
    np.random.seed(np.random.get_state()[1][0] + worker_id)

print(worker_init_fn(1))

None


In [4]:
import sys
from sklearn.datasets import fetch_olivetti_faces

data = fetch_olivetti_faces()
X, y = data.data, data.target

from sklearn.model_selection import train_test_split
# split_ratio = 0.1 としているので，訓練データ対テストデータが 8:2 になります
split_ratio = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=split_ratio, 
                                                    stratify=y,
                                                    random_state=0)
print(f'X_train 訓練画像のサイズ: {X_train.shape}')
print(f'y_train 教師信号データのサイズ: {y_train.shape}')

X_train 訓練画像のサイズ: (320, 4096)
y_train 教師信号データのサイズ: (320,)


In [5]:
# データセットの作成
class Mydataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        feature = self.X[index]
        label = self.y[index]
        return feature, label


X_ = torch.tensor(X_train).float()
X_ = torch.reshape(torch.tensor(X_train).float(), (-1,1,64,64))
y_ = torch.tensor(y_train).long()
Xtest_ = torch.tensor(X_test).float().reshape(-1,1,64,64)
ytest_ = torch.tensor(y_test).long()

#train_dataset = Mydataset(train_X, train_y)
#test_dataset = Mydataset(test_X, test_y)
train_dataset = Mydataset(X_, y_)
test_dataset = Mydataset(Xtest_, ytest_)

# データローダーの作成
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=128,   # バッチサイズ
                                               shuffle=True,     # データシャッフル
                                               num_workers=0,    # 高速化
                                               pin_memory=True,  # 高速化
                                               worker_init_fn=worker_init_fn
                                              )
test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=128,
                                              shuffle=False,
                                              num_workers=0,
                                              pin_memory=True,
                                              worker_init_fn=worker_init_fn
                                             )

In [6]:
# モデルの定義
class Mymodel_olivetti(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Sequential(nn.Conv2d(in_channels=1, 
                                                   out_channels=16, 
                                                   kernel_size=3, 
                                                   stride=1,
                                                   padding=1,
                                                  ),
                                         nn.BatchNorm2d(16),
                                         nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
                                         nn.ReLU())
        
        self.conv2 = torch.nn.Sequential(nn.Conv2d(in_channels=16, 
                                                   out_channels=16, 
                                                   kernel_size=3, 
                                                   stride=1,
                                                   padding=1,
                                                  ),
                                         nn.BatchNorm2d(16),
                                         nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
                                         nn.ReLU())

        self.fc1 = nn.Linear(16 * 64 * 64, 100)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = torch.nn.Linear(100, 40)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
model = Mymodel_olivetti()
from torchsummary import summary
#summary(model, input_size=(1, 64, 64))

x = torch.rand([1,1,64,64])
y = model.forward(x)
torchviz.make_dot(y.mean(), params=dict(model.named_parameters()))

In [8]:
# モデル・損失関数・最適化アルゴリスムの設定
model = Mymodel_olivetti().to(device)
loss_f = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model.parameters(), lr=0.01)
optimizer = optim.Adam(model.parameters(), weight_decay=0.01)

# モデル訓練関数
def train_(model, train_loader, test_loader):
    train_losses, test_losses = [], []

    # Train loop ----------------------------
    model.train()  # 学習モードをオン
    for data, label in train_loader:
        # device (CPU, GPU) への転送
        data, label = data.to(device), label.to(device)
        optimizer.zero_grad() # 1. 勾配のリセット
        output = model(data) # 2. 推論
        loss = loss_f(output, label) # 3. 誤差計算
        loss.backward()  # 4. 誤差逆伝播
        optimizer.step() # 5. パラメータ更新
        train_losses.append(loss.item()) # train_lossの取得

    # Test(val) loop ----------------------------
    model.eval()  # 学習モードをオフ
    with torch.no_grad():  # 勾配を計算なし
        for data, label in test_loader:
            data, label = data.to(device), label.to(device)
            output = model(data)
            loss = loss_f(output, label)
            test_losses.append(loss.item())

    return np.mean(train_losses), np.mean(test_losses)

In [ ]:
# 訓練の実行
epoch = 20
train_loss = []
test_loss = []

for epoch in tqdm(range(epoch)):
    _train_loss, _test_loss = train_(model, train_dataloader, test_dataloader)
    train_loss.append(_train_loss)
    test_loss.append(_test_loss)


# 学習進行状況の描画
plt.plot(train_loss, label='訓練損失')
plt.plot(test_loss, label='テスト損失')
plt.legend()